# data processing of unlabeled data (no clinvar)

In [7]:
import pandas as pd
'''

### TO DO for MK:
* Once the data matches the columns, then:
 - check the order 
 - run the bash command below to get the features for the dataframe,
 - pickle it, then 
 - move to the model prediction. 


PARTIAL! Feature set 4 selected for 1000 unlabeled examples of first 4 chromosomes.
--> columns after 359 do not exist in the input data, will probably rerun with a proper feature set. 

### MK used:
for i in 1 2 3 4; do python3 keep_relevant_headers.py \
../data/chr1_4_nolables_gene/chr$i\_1000\_nolabels.tsv \
../data/chr1_4_nolables_gene/chr$i.nolables.out_set5.tsv \
--cols chr,pos,ref,alt,aapos,codonpos,Polyphen2_HVAR_pred,\
GenoCanyon_score,LINSIGHT,clinvar_clnsig,RVIS_ExAC,ExAC_cnv.score,SORVA_LOF_MAF0.005_HetOrHom; done

'''#chr4.nolables.out_set5.tsv
#chr1_4_nolables_gene/chr1.nolables.out_set5.tsv

df_all = pd.read_csv('../../../data/chr1_4_nolables_gene/chr1.nolables.out_set5.tsv', sep='\t', encoding = "UTF-8")
df_all = df_all.set_index(['chr','pos', 'ref', 'alt']) #index by identifier

for c in range (2,5):
    #print(c)
    path = '../../../data/chr1_4_nolables_gene/chr' + str(c) + '.nolables.out_set5.tsv'
    df = pd.read_csv(path, sep='\t', encoding = "UTF-8")
    df = df.set_index(['chr','pos', 'ref', 'alt']) #index by identifier
    df_all= df_all.append(df)
    
print('concat shape', df_all.shape)

df = df_all
print("shape before", df.shape)
#print(df.head)
df = df.replace({'.': 0})  ## FIXME: atm replace with nan and drop: later find a way to impute, maybe 0 but not sure
#print("SUM", pd.isnull(df).sum())  ## Use this to see how many elements have nan 
df = df.dropna()
print("shape after dropping all data with score = '.'", df.shape)
print(df.head())

## still contains some mixed labels, see them by: df[~df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]

### prior to remapping of clinical significance labels to only pathogenic/benign, check out the distirbutions:
#print("TOTAL SUM OF clinsig: \n", df['clinvar_clnsig'].value_counts())
#print("PERCENT OF clinsig: \n", df['clinvar_clnsig'].value_counts(normalize=True))
#print("CLINVAR LABELS \n", df.groupby('clinvar_clnsig').count())

## Map labels to 1/0 and remove Unkn
#clinvar_labels={
#    'Pathogenic': 1,
#    'Likely_pathogenic': 1,
#    'Pathogenic/Likely_pathogenic': 1,
#    'Benign/Likely_benign': 2,
#    'Likely_benign': 2,
#    'Benign': 2,
#    'Uncertain_significance': 0
#}

features_mapping = {
    'Polyphen2_HVAR_pred': {
        'D': -2,
        'P': -1, 
        'B': 1
    }
}

df = df.replace(features_mapping)

#df = df[df['clinvar_clnsig'].isin(list(clinvar_labels.keys()))]  # best to drop everything that isn't in the dict
#df['Problematic'] = df['clinvar_clnsig'].map(clinvar_labels)

#df= df.drop(columns=['clinvar_clnsig'])#, 'clinvar_id'])  #tbh, I don't know why I kept them. consider adding to index
#print("shape after dropping mixed labels", df.shape)

### pickle to reuse dataframe:
df.to_pickle("../pickled/chr1-4_nolabels.pkl")

#print("PROBLEMATIC LABEL DISTRIBUTION \n", df.groupby('Problematic').count())
#print("TOTAL SUM OF LABELS: \n", df['Problematic'].value_counts())
#print("PERCENT OF LABELS: \n", df['Problematic'].value_counts(normalize=True))

concat shape (3995, 9)
shape before (3995, 9)
shape after dropping all data with score = '.' (3988, 9)
                    aapos codonpos Polyphen2_HVAR_pred     GenoCanyon_score  \
chr pos    ref alt                                                            
1   65565  A   G        1        1                   0  2.01693589783425E-5   
    69453  G   C      121        3                   0  1.61590270936468E-5   
    69887  T   G      266        2                   0  4.18119972739408E-6   
    451053 C   G      209        2                   0                    0   
    451486 T   G       65        1                   0                    0   

                    LINSIGHT  clinvar_clnsig RVIS_ExAC ExAC_cnv.score  \
chr pos    ref alt                                                      
1   65565  A   G    0.206807               0         0              0   
    69453  G   C           0               0         0              0   
    69887  T   G           0               0       